In [1]:
"""

啟用伺服器基本樣板

"""
from gensim.models import word2vec
import sys

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./secretFile.txt",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channelAccessToken"))
handler = WebhookHandler(secretFileContentJson.get("channelSecret"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [2]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單

reply_message_list = [
    TextSendMessage(text="歡迎來到!好菜底家~")
]

text_reply_message1 = TextSendMessage(text="請輸入想替換食材")

In [3]:
template_message_dict = {
    "[::text:]食材替換":text_reply_message1
}

In [4]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("richmenuId")
    linkResult=line_bot_api.link_rich_menu_to_user(secretFileContentJson["userId"], linkRichMenuId)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="歡迎來到!好菜底家~")
    )

In [5]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('::text:')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    '''
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="請點擊菜單上圖面")
        )
    '''
    if(event.message.text.find('::switch:')!= -1):
        model = word2vec.Word2Vec.load("word2vec.model")
        switch = model.wv.most_similar(event.message.text.replace('[::switch:]',''))
        switch_message = '%s %s %s' %(switch[0][0],switch[1][0],switch[2][0])
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=switch_message)
        )
        #print(event.message.text.replace('[::switch:]',''))
        

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.20.0.5 - - [16/Aug/2020 11:22:35] "POST / HTTP/1.1" 200 -
172.20.0.5 - - [16/Aug/2020 11:22:37] "POST / HTTP/1.1" 200 -
172.20.0.5 - - [16/Aug/2020 11:22:45] "POST / HTTP/1.1" 200 -
172.20.0.5 - - [16/Aug/2020 11:22:50] "POST / HTTP/1.1" 200 -
